In [7]:
import os
import json
import keyring
import requests
import traceback
import pandas as pd
from OsOps import Ops
from spotipy import client, util
from collections import namedtuple

In [9]:
# AUTHENTICATE (POST), FROM CREDS STORED IN WIN CRED MGR
user_id, sp_cid, sp_sec = ( 
    keyring.get_password( service, "" )
    for service in [
        "spottyPie_user_id",
        "spottyPie_clientID", 
        "spottyPie_clientSecret"] )

if None in [user_id, sp_cid, sp_sec]: print( "Credential not set locally" )

In [21]:
ops = Ops()

# standard API authenticate (not used)
def getAuthResponse():
    try: return (
        requests.post( 'https://accounts.spotify.com/api/token', 
            { 'grant_type': 'client_credentials',
                'client_id': sp_cid,
                'client_secret': sp_sec, }) )
    except Exception as exc: print( 
        f"\n[ ERRR ] {exc.__class__}"
        f"\n[ DSTR ] {exc.__doc__}"
        f"\n[ CTXT ] {exc.__context__}"
        f"\n{ '='*79 }"
        f"\n\n{traceback.format_exc()}" )

# get authenticated spotipy object
def auth_SpotPy():
    token = util.prompt_for_user_token(
        username= user_id, 
        scope= " ".join( [
            "playlist-read-private",
            "playlist-modify-private",
            "user-library-modify",
            "user-read-private" ]),
        client_id= sp_cid, 
        client_secret= sp_sec, 
        redirect_uri = "http://example.com/")

    return client.Spotify(token)

spot = auth_SpotPy()

In [12]:
# full albums from tracks in any list, sorted by largest album

def createNewPL( pl_name="New_PL", stamp=True, ):
    try: return spot.user_playlist_create( 
        user= user_id, 
        name= f"{pl_name}_{ops.dtStamp()}" if stamp else pl_name, 
        public=False )
    except Exception as e: return f"EXC createNewPL:\n{type(e).__name__}\n{e}"

def albsFromPList( pl_id ):
    track_list = spot.user_playlist_tracks( user=user_id, playlist_id=pl_id, )
    return { i : {
        "alb_id" : item['track']['album']['id'], 
        "tracks" : item['track']['album']['total_tracks'],
        "dct" : item, } 
        for i, item in enumerate( track_list['items']) }

def getAllTrackIDs( listDict ):
    Alb = namedtuple( "Alb", [ "alb_id", "siz", "dct" ] )
    albsSorted = sorted( [ 
        Alb( d["alb_id"], d['tracks'], d["dct"] )
        for _, d in listDict.items() ],
        key = lambda el: el.siz, reverse=True )
    return [ item['id']
        for alb in albsSorted
        for item in spot.album_tracks( alb.alb_id )['items'] ]

def addTracksByID( idList, destination ):
    def yieldSegments(li, size):
        for i in range(0, len(li), size): yield li[ i:i + size ]
    for seg in yieldSegments(idList, 100):  # max 100
        spot.user_playlist_add_tracks(
            user_id, 
            playlist_id=destination, 
            tracks=seg)
            
created_pl = createNewPL( pl_name="rrad" )
# created_pl = createNewPL( pl_name="dsco" )
rradDct = albsFromPList( "37i9dQZEVXbuX4MySjIacD" ) # release radar 
# dscoDct = albsFromPList( "37i9dQZEVXcXssf47BUM1F" ) # dscov weekly 
allTrackIDs = getAllTrackIDs( rradDct )
# allTrackIDs = getAllTrackIDs( dscoDct )
addTracksByID( allTrackIDs, created_pl['id'] )
# addTracksByID( allTrackIDs, created_pl['id'] )

In [24]:
# add: shuffle playlist by id; 
    # shuffle must ensure maximum distance between songs of the same album
        # THEN/AND maximum distance between songs of same artist
        # AND NOT alphabetical or album order
track_dct = spot.user_playlist_tracks( user=user_id, playlist_id="2gnGX8RzMnqYbVYZuUnkPe", )

In [49]:
# Create counted "groups of same album" 
# get required distance for items of each group (total size/count)

album_groups = {}
len_items = 0

for item in track_dct["items"]:
    
    len_items +=1
    
    alb_id = item["track"]["album"]["id"]
    trk_id = item["track"]["id"]
    
    if alb_id in album_groups:
        album_groups[alb_id]["items"].add(trk_id)
        album_groups[alb_id]["count"] += 1
        
    else: album_groups[alb_id] = { "items": set([trk_id]), "count": 1 }

for k, v in album_groups.items(): print( f"{k}, {v['count']}" )

28VtIxXxSnHaQFKCnKN0P5, 8
5NYiyNqu9Joe1qtgJN01w3, 10
3FvnZykNfyYuJ94zL46HrK, 9
6vw8dbzcZ0a4btA4lGBLCL, 1
10aiDpdFGyfCFEcqpx6XTq, 1
0kbAsmfSMFpq8wfPki1kQj, 1
09Df7mUZBQwbDYgvE0t30r, 1
1pIwUzeY7bzkKQJrqA4gHQ, 1
3naxO4ZMrfJqEpEKjkVvlH, 1
57ttk3xzHMWLr6CGrEa8F3, 3
57y7GwBbU7iWlVt3fO4yQA, 11
6Oa2B45zimQ8lfbknds32G, 1
3ArbhwwokDjBYwyJYIMEcZ, 2
6A0IarahNWMnEcJ1Q6sn7d, 2
5tD7xrX0TjRHaGCbSFDz3D, 1
1mdlZytWHqyITkGx18A4R0, 2
0YFhQCQPheByANPPcAmfHn, 1
0BWJXExyO22HPW2ja2SWCb, 1
2pANu4qucnliJuRR94eZSV, 1
218CJKDCszsQQj7Amk7vIu, 1
2DOiha5oI19Dmw5M9ryHD8, 1
1rDjSDjjzVfpnsb3GgREFf, 1
2pbga7SePsAKlYs3iKlWCF, 2
3eHcKAlikW4c62i55ZGu0c, 2
3WvQpufOsPzkZvcSuynCf3, 4
4Y0PrDckfFKxKaVXsscDLB, 2
1jL9mDhM5cMAwJD9brYyW5, 1
4FF4II3GGoTuYRW3zenamA, 2
6XBKxM9rbzyOBP6gpE0qhd, 1
0c86awAGyX0pSNQYNwXKsy, 1
3FsCGJY0JqSxvgezoMWYzF, 1


In [50]:
# import math

In [ ]:
from collections import Counter

In [258]:
# to shuffle by other attribs than album, segment separately into Counter

alb_id_list = [ "B", "A", "A", "C", "B", "A", "A", ]

list_size = len(alb_id_list)
alb_id_groups = Counter(alb_id_list).most_common()

# INITIALIZE
# values for minimum-frequency counter, store initial to reset when decr. to 0

alb_id_mfCounts_orig = { alb_id : int( list_size / count ) 
    for alb_id, count in alb_id_groups }

alb_id_mfCounts = alb_id_mfCounts_orig.copy()

# create new list, adding from first group at curr_g_size
# testing simply with g_posit
g_posit = 0
new_alb_id_list = [ alb_id_groups[g_posit][0] ]


In [259]:
# main pattern
# Check any mf_count==0 in order: None, 

def check_mf_count():
    for alb_id, mf_counter in alb_id_mfCounts.items(): 
        if mf_counter<=0: return alb_id
    return None

while True:
    if ( insert_id := check_mf_count() ):
        # print( f"mf_count for {insert_id} reached <=0")
        new_alb_id_list.append( insert_id )
        
        alb_id_mfCounts = { alb_id : count-1 
            if alb_id != insert_id
            else alb_id_mfCounts_orig[insert_id]
            for alb_id, count in alb_id_mfCounts.items() }
            
    else: 
        # Check any n_size==curr_n_size: None
        # replaced with simply "next in list"
        g_posit += 1
        # if g_posit+1 >= len(alb_id_groups): ("finito")
        try: new_alb_id_list.append( alb_id_groups[g_posit][0] )
        except IndexError: 
            print(f"{new_alb_id_list=}" )
            break
        alb_id_mfCounts = { alb_id : count-1 
            for alb_id, count in alb_id_mfCounts.items() }


new_alb_id_list=['A', 'B', 'A', 'C', 'A', 'B', 'A']


In [ ]:
# sort albums by largest distance
# draw first item, and insert at first possible required position for group
    # that position-set now belongs to items of that type;
        # insert all items across same position-set
# insert according to required position
# 

In [17]:
from collections import Counter

In [19]:
# get total length of list
list_len = len(track_ids)
# dict with count of each album id
# Counter(track_ids)

In [ ]:


allTrackIDs = getAllTrackIDs( rradDct )

def shuffleTracks():
    pass

addTracksByID( allTrackIDs, created_pl['id'] )


In [11]:
# get library as dataframe ( all playlists, all tracks ) ~10min
def getPlaylists( asDict=False ):
    '''folder structure not available through API as at 221224'''
    incrmt = 50  # max 50
    offset = 0
    plistsDct = {}
    while True:
        newItemDcts = spot.current_user_playlists( incrmt, offset)["items"]
        plistsDct.update( { iDct["id"]: iDct for iDct in newItemDcts } )
        if len(newItemDcts) < incrmt: 
            return pd.DataFrame( plistsDct ).T if not asDict else plistsDct
        else: offset += incrmt

def getTracks( id ):
    incrmt = 100  # max 100
    offset = 0
    tracks = {}
    while True:
        items = spot.user_playlist_tracks( 
            user=user_id, playlist_id=id, limit=incrmt, offset=offset )["items"]
        tracks.update( { i["track"]["id"]: i for i in items } )
        if len(items) < incrmt: return tracks
        else: offset += incrmt

# df_pLists = getPlaylists()
# df_pLists["tracklist"] = df_pLists["id"].apply( lambda id: getAllTracks( id ) )
# ops.storePKL( df_pLists, "df_pLists", os.getcwd() )

In [10]:
plDct = getPlaylists( asDict=True )
for id, dct in plDct.items(): dct["tracks"].update({ "list": getTracks( id )})
ops.storePKL( plDct, "plDct", os.getcwd() )

In [9]:
# PODCASTS: Latest [n] eps from shows in playlist of episodes
def latestNEps( pl_id, n=3 ):
    tracks = spot.user_playlist_tracks( user=user_id, 
        playlist_id=pl_id, )

    showIDs = set( artist['id']
        for item in tracks["items"]
        for artist in item["track"]["artists"] 
        if artist["type"]=="show" )  # skip non-podcast tracks

    epIDs = []
    for showID in showIDs:
        show_items = spot.show_episodes(showID)['items']
        eps_recent = sorted( [ ( item["release_date"], item["id"] )
            for item in show_items ], key= lambda i: i[0], reverse=True )
        epIDs.extend( f'spotify:episode:{id}' for _, id in eps_recent[:n] )
        
    return epIDs

# new_pCasts_PL = createNewPL( pl_name="PCST" )
# epIDs = latestNEps( "2PFeIO0B0DtenFmGKbzYvg", n=3 )
# addTracksByID( epIDs, new_pCasts_PL['id'] )